In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [57]:
budget_df = pd.read_csv('Phase 1 Project Data/tn.movie_budgets.csv')

In [58]:
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


TN seems to be a better dataset compared to BOM since it has more data and has a budget column which will allow you to calculate profit. Also there are no missing values 

In [59]:
#Convert columns 3-5 into integers. Create profit column

budget_df['production_budget'] = budget_df['production_budget'].str.replace(',','').str.replace('$','').astype(int)
budget_df['domestic_gross'] = budget_df['domestic_gross'].str.replace(',','').str.replace('$','').astype(int)
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].str.replace(',','').str.replace('$','').astype(int)

In [67]:
#create profit. Split out month from the release date. Create season
budget_df['profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']
budget_df['release_date'] = pd.to_datetime(budget_df['release_date'])
budget_df['release_month'] = budget_df['release_date'].dt.month
budget_df['quarter'] = budget_df['release_month'].apply(lambda x:

SyntaxError: unexpected EOF while parsing (<ipython-input-67-cd2bec3e99cd>, line 5)

12    745
10    573
8     496
9     493
11    486
6     479
3     470
4     454
7     440
5     407
2     392
1     347
Name: release_month, dtype: int64